# Автосервис. Описание проекта

Нужно построить модель для определения стоимости.

**Заказчику важны:**
- качество предсказания;
- скорость предсказания;
- время обучения.

### Признаки
- `DateCrawled` — дата скачивания анкеты из базы
- `VehicleType` — тип автомобильного кузова
- `RegistrationYear` — год регистрации автомобиля
- `Gearbox` — тип коробки передач
- `Power` — мощность (л. с.)
- `Model` — модель автомобиля
- `Kilometer` — пробег (км)
- `RegistrationMonth` — месяц регистрации автомобиля
- `FuelType` — тип топлива
- `Brand` — марка автомобиля
- `NotRepaired` — была машина в ремонте или нет
- `DateCreated` — дата создания анкеты
- `NumberOfPictures` — количество фотографий автомобиля
- `PostalCode` — почтовый индекс владельца анкеты (пользователя)
- `LastSeen` — дата последней активности пользователя

**Целевой признак**
- `Price` — цена (евро)

In [26]:
# Импорт стандартной библиотеки
import pandas as pd
import numpy as np
import datetime
import time
import lightgbm
import catboost
import re

# Сторонний импорт
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyClassifier
from sklearn.metrics import mean_squared_error
from fast_ml.model_development import train_valid_test_split
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from matplotlib import pyplot

import warnings
warnings.filterwarnings('ignore')


In [27]:
data.info()
data = data.sort_values(['RegistrationYear', 'RegistrationMonth',
                         'Price', 'Power', 'Kilometer'])
display(data)
print()
display(data.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
78128,2016-03-05 23:36:25,0,NaN,1000,NaN,0,NaN,5000,0,NaN,sonstige_autos,NaN,2016-03-05 00:00:00,0,59387,2016-04-06 01:15:49
230741,2016-03-19 12:37:35,0,NaN,1000,NaN,0,NaN,5000,0,NaN,sonstige_autos,NaN,2016-03-19 00:00:00,0,36304,2016-03-19 12:37:35
242233,2016-03-14 20:54:38,0,NaN,1000,NaN,0,NaN,5000,0,NaN,mercedes_benz,NaN,2016-03-14 00:00:00,0,53783,2016-03-21 17:49:41
119442,2016-03-18 10:37:00,1,NaN,1000,NaN,1000,3er,5000,0,NaN,bmw,NaN,2016-03-18 00:00:00,0,94086,2016-04-05 22:16:13
348830,2016-03-22 00:38:15,1,NaN,1000,NaN,1000,NaN,150000,0,NaN,sonstige_autos,NaN,2016-03-21 00:00:00,0,41472,2016-04-05 14:18:01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28965,2016-04-04 22:54:47,18000,NaN,9999,NaN,0,a_klasse,10000,0,petrol,mercedes_benz,NaN,2016-04-04 00:00:00,0,51379,2016-04-07 02:44:52
301279,2016-03-20 15:06:24,19000,NaN,9999,NaN,0,transporter,10000,0,NaN,volkswagen,NaN,2016-03-20 00:00:00,0,60439,2016-04-07 00:44:47
306578,2016-03-22 15:50:24,350,NaN,9999,NaN,0,kaefer,10000,1,NaN,volkswagen,NaN,2016-03-22 00:00:00,0,27432,2016-04-06 05:45:40
28390,2016-04-05 08:57:08,799,NaN,9999,NaN,0,3er,10000,4,petrol,bmw,NaN,2016-04-05 00:00:00,0,72116,2016-04-07 12:17:50


,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


- **Обнаружено много пропусков. Например, месяц регитрации много где записан нулем. Удалять 10% данных - это вряд ли хорошая идея. Поэтому просто не трогаем это значение.** 
- **Столбец с числом фотографий равен нулю.** 
- **Данные о пробеге в большенстве равен 15 000 км. Мало пользы от такого признака.** 
- **В столбце с целевым признаком, с ценой, явная аномалия. Их около 12 тысяч строк, это примерно 3,4% данных. Может ли машина стоить 0, 5, 15, 50 евро? Как-то осмысленно их заполнить нельзя, удалить тоже. Совершенно неясно, какая реальная минимальная цена машины должна быть. Но 12 тысяч бесплатных - неправдоподобно. Если бы они были битые - но не факт, в столбце "NotRepaired" разные значения в этих строках. Как бы то ни было, трогать таргет не мы будем, так как если изменить таргет - это будет уже другая задача.**
- **Обнаружены аномалии после фильтрации признаков по датам. Например, дата регистрации автомобиля 1000-9999.**

## Предобработка. Преобразование признаков. Заполнение пропусков. Разделение на выборки

In [28]:
# Преобразуем признаки даты в float
dates = ['DateCrawled', 'DateCreated', 'LastSeen']
for date in dates:
    data[date] = (pd.to_datetime(data[date], format='%Y-%m-%d')
                    .apply(lambda x: x.timestamp()))

data = data.sort_values(['RegistrationYear', 'RegistrationMonth',
                         'DateCreated', 'LastSeen',
                         'DateCrawled', 'Price',
                         'Power', 'Kilometer'])

# Заполнение пропусков. Ставим заглушку
columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'NotRepaired']
for col in columns:
    data[col] = data[col].fillna('unknown')

# Исправим на змеиный метод написания
data.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', name)
                  .lower() for name in data.columns]

# Убираем сроки где цена за авто равна нулю.
data = data[data['price'] > 0]

# Проверяем что получилось
display(data)

# Преобразуем категориальные признаки методом OHE
data = pd.get_dummies(data, drop_first=True)

# Разделение на выборки
X_train, y_train, X_valid, y_valid, X_test, y_test = train_valid_test_split(data, target = 'price',  train_size=0.8, 
                                                                            valid_size=0.1,  test_size=0.1)


,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
256532,1.457200e+09,12500,unknown,1000,unknown,200,golf,5000,0,unknown,volkswagen,unknown,1.456618e+09,0,75378,1.460031e+09
135865,1.457363e+09,16500,unknown,1000,unknown,0,unknown,5000,0,unknown,sonstige_autos,unknown,1.457309e+09,0,23879,1.457551e+09
213499,1.457439e+09,380,unknown,1000,unknown,0,6er,5000,0,unknown,bmw,unknown,1.457395e+09,0,35102,1.459902e+09
55605,1.457639e+09,500,unknown,1000,unknown,0,unknown,5000,0,unknown,citroen,yes,1.457568e+09,0,24811,1.459914e+09
60017,1.457613e+09,80,unknown,1000,unknown,0,unknown,5000,0,unknown,volkswagen,unknown,1.457568e+09,0,93107,1.460004e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268091,1.459386e+09,150,unknown,9999,unknown,0,unknown,10000,0,unknown,sonstige_autos,unknown,1.459296e+09,0,28870,1.459567e+09
149151,1.459623e+09,400,unknown,9999,unknown,0,unknown,10000,0,unknown,sonstige_autos,unknown,1.459555e+09,0,26441,1.459623e+09
28965,1.459810e+09,18000,unknown,9999,unknown,0,a_klasse,10000,0,petrol,mercedes_benz,unknown,1.459728e+09,0,51379,1.459997e+09
306578,1.458662e+09,350,unknown,9999,unknown,0,kaefer,10000,1,unknown,volkswagen,unknown,1.458605e+09,0,27432,1.459922e+09


## Исследование моделей

Чтобы усилить исследование, не ограничимся градиентным бустингом. Опробуем более простые модели — иногда они работают лучше. Это редкие случаи, которые легко пропустить, если всегда применять только бустинг. Поэкспериментируем и сравним характеристики моделей: скорость работы, точность результата.

### Простые модели

In [29]:
%%time
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_valid)
result = mean_squared_error(y_valid, predictions)**.5
print("RMSE LinearRegression:", result)


RMSE LinearRegression: 3071.6499767989353
Wall time: 5.9 s


In [30]:
%%time
best_model = None
best_result = 10000
best_depth = 0
for depth in range(1, 6):
    model = DecisionTreeRegressor(random_state=12345, max_depth=depth)
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    result = mean_squared_error(y_valid, predictions) ** 0.5
    if result < best_result:
        best_model = model
        best_result = result
        best_depth = depth

print("RMSE DecisionTreeRegressor:", best_result,
      "Глубина дерева:", best_depth)


RMSE DecisionTreeRegressor: 2483.2049864256182 Глубина дерева: 5
Wall time: 7.07 s


In [31]:
%%time
best_model = None
best_result = 10000
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range(1, 11):
        model = RandomForestRegressor(random_state=12345, n_estimators=est,
                                      max_depth=depth)
        model.fit(X_train, y_train)
        predictions = model.predict(X_valid)
        result = mean_squared_error(y_valid, predictions) ** .5
        if result < best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth

print("RMSE RandomForestRegressor:", best_result,
      "Количество деревьев:", best_est,
      "Максимальная глубина:", best_depth)


RMSE RandomForestRegressor: 1953.6016106317122 Количество деревьев: 50 Максимальная глубина: 10
Wall time: 35min 55s


### Модели градиентного бустинга

In [32]:
def result(estimator, rf_grid):
    model = GridSearchCV(estimator, rf_grid,
                         scoring='neg_root_mean_squared_error',
                         n_jobs=5, cv=3)
    
    result = model.fit(X_train, y_train)

    print(f'{str(estimator)}. Best Hyperparameters: %s' % result.best_params_)
    y_pred = model.predict(X_valid)
    predictions = [round(value) for value in y_pred]
    
    print(f'{str(estimator)}. RMSE: %.2f' %
          mean_squared_error(y_valid, predictions)**.5)


In [33]:
%%time
rf_grid = {'n_estimators': [200, 300, 500, 700, 1000],
           "min_child_weight": [1, 3, 5, 10, 25],
           'max_depth': [3, 4, 7, 10, 25, 50]
           }
result(LGBMRegressor(force_row_wise=True), rf_grid)


LGBMRegressor(force_row_wise=True). Best Hyperparameters: {'max_depth': 25, 'min_child_weight': 1, 'n_estimators': 1000}
LGBMRegressor(force_row_wise=True). RMSE: 1571.84
Wall time: 11min 34s


In [34]:
%%time
param = {'iterations': [3],
         'max_depth': [10, 25, 50, 75, 100, 200],
         'min_child_samples': [1, 2, 3]
         }
result(CatBoostRegressor(), param)


Learning rate set to 0.5
0:	learn: 3101.7504414	total: 173ms	remaining: 345ms
1:	learn: 2477.9526692	total: 210ms	remaining: 105ms
2:	learn: 2215.2691658	total: 244ms	remaining: 0us
<catboost.core.CatBoostRegressor object at 0x000001C2916CB550>. Best Hyperparameters: {'iterations': 3, 'max_depth': 10, 'min_child_samples': 1}
<catboost.core.CatBoostRegressor object at 0x000001C2916CB550>. RMSE: 2214.34
Wall time: 32.1 s


### Итак, можно выделить:
- качество моделей с градиентным бустингом выше чем у простых. 
- лучшее качество `LightGBM`.
- скорость `CatBoost` такая же высокая как у простых моделей, но качество выше.

## Проверим CatBoost на тестовой выборке

In [37]:
%%time

model = CatBoostRegressor(iterations=3, max_depth=10, min_child_samples=1)
model.fit(X_train, y_train)

predictions_train = model.predict(X_train)
predictions_test = model.predict(X_test)

clf = DummyClassifier(strategy='most_frequent', random_state=12345)
clf.fit(X_train, y_train)
print("DummyClassifier: %.2f" % clf.score(X_test, y_test))
print("Наилучшая модель - CatBoost.")
print("RMSE на обучающей выборке: %.2f" %
      mean_squared_error(y_train, predictions_train)**.5)
print("RMSE на тестовой выборке: %.2f" %
      mean_squared_error(y_test, predictions_test)**.5)


Learning rate set to 0.5
0:	learn: 3101.7504414	total: 34.9ms	remaining: 69.7ms
1:	learn: 2477.9526692	total: 67.2ms	remaining: 33.6ms
2:	learn: 2215.2691658	total: 99.4ms	remaining: 0us
DummyClassifier: 0.02
Наилучшая модель - CatBoost.
RMSE на обучающей выборке: 2215.27
RMSE на тестовой выборке: 2249.72
Wall time: 621 ms


# Заключение: лучшая модель - `CatBoost`

Применили простые модели, так и с градиентным бустингом. Поэкспериментировали и сравнили характеристики моделей: скорость работы, точность результата. 

Исходя из тех параметров которыми пришлось ограничится для того что бы сдать проект в срок, лучший результат по качеству - модель градинетного бустинга `LightGBM`. 
Скорость `LightGBM`, в рамках этого эскперимента, уступает большпинству других моделей кроме `RandomForestRegressor`, но качество результата заметно выше всех остальных. 

Относительно скорости можно выделить `CatBoost` - имеет скорость простых моделей и более лучшие результаты, но по качеству уступает `LightGBM`.

**Но т.к для заказчика важна не только точность, но и скорость - отдаем предпочтение второй модели. Лучшая модель: `CatBoost`.**